In [22]:
import pandas as pd
import numpy as np

ydata = pd.read_csv("./data/data2.csv")
lydata = ydata.copy()
lydata = np.log(lydata + 1e-12)
gen=9.8

In [8]:
y = ydata["HeatFlux"]
x = ydata[["Superheat", "g", "SurfaceTension", "p"]]

In [13]:
def fun(x, n1, n2, n3, n4, n5):
    y = n1 * np.power(x["Superheat"], n2) * np.power((ydata["g"]+n4*9.8*ydata["SurfaceTension"]), n3) * np.power(ydata["p"], n5)
    return y



In [16]:
# Regression
from scipy import optimize
n = optimize.curve_fit(fun, x, y)[0]

In [17]:
def pred_true(n, label):
    # Calculating y_pred
    n = n.reshape(1,-1)
    y_pred = np.log(n[:, 0]) + n[:, 1] * lydata["Superheat"] + n[:, 2] * np.log(ydata["g"] + n[:, 3]*gen*ydata["SurfaceTension"]) + n[:, 4]*lydata["p"]
    y_pred = np.exp(y_pred)
    y_true = ydata["HeatFlux"]
    # Plotting
    plt.scatter(y_pred, y_true, label=label)

# Deviation and other statistic results
def stat(n):
    n = n.reshape(1,-1)
    y_pred = np.exp(np.log(n[:, 0]) + n[:, 1] * lydata["Superheat"] + n[:, 2] * np.log(ydata["g"] + n[:, 3]*gen*ydata["SurfaceTension"]) + n[:, 4]*lydata["p"])
    y_true = ydata["HeatFlux"]
    relErr = (y_pred-y_true) / y_true
    mean = np.sqrt((np.mean(relErr * relErr) / relErr.shape[0]))
    dev = np.std(relErr)
    unacc_rate = np.where((relErr > 0.1)| (relErr < -0.1))[0].shape[0] / relErr.shape[0]
    print("RMSError: %f, RMSDeviation: %f, Unacceptable Rate: %f"%(mean, dev, unacc_rate))
    return mean, dev, unacc_rate

In [23]:
stat(n)

RMSError: 0.017526, RMSDeviation: 0.148538, Unacceptable Rate: 0.363636


(0.01752612658279503, 0.14853762485137778, 0.36363636363636365)